# Negative sampling

In [1]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sklearn.metrics       as Metrics
import pandas                as pd
import matplotlib            as mpl
import random
from calendar import monthrange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


# import folium  
# from folium.plugins import HeatMap
# import branca

In [2]:
import json
import requests
import os
import calendar
import datetime

In [3]:
# set matplotlib defaults
%matplotlib inline
sns.set()
plt.rcParams["figure.figsize"] = (15,6)
mpl.rc('axes', labelsize=18)
mpl.rc('xtick', labelsize=16)
mpl.rc('ytick', labelsize=16)
plt.rc('figure', titlesize=18)
plt.style.use('seaborn-darkgrid')

## Data loading and preparation

# MODELING

In [14]:
data_folder = '../data'
data_15 = os.path.join(data_folder, 'dataset_2015_with_negatives.csv')
data_16 = os.path.join(data_folder, 'dataset_2016_with_negatives.csv')
data_17 = os.path.join(data_folder, 'dataset_2017_with_negatives.csv')
data_18 = os.path.join(data_folder, 'dataset_2018_with_negatives.csv')
data_19 = os.path.join(data_folder, 'dataset_2019_with_negatives.csv')
df_15 = pd.read_csv(data_15,sep=',')
df_16 = pd.read_csv(data_16,sep=',')
df_17 = pd.read_csv(data_17,sep=',')
df_18 = pd.read_csv(data_18,sep=',')
df_19 = pd.read_csv(data_19,sep=',')
# df.head()

/home/juan/anaconda3/envs/ds4a/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/juan/anaconda3/envs/ds4a/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10,11,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
df_15

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,...,windSpeed,cloudCover,uvIndex,visibility,windBearing,windGust,pressure,precipType,ozone,precipAccumulation
0,-74.023482,4.766878,2015-10-17 21:00:00,2015-10-17,2015,10,17,290,21,KR 3-CL 189C 02,...,3.10,0.44,2.0,10.003,90.0,NaN,NaN,NaN,NaN,NaN
1,-74.072893,4.598232,2015-02-17 08:10:00,2015-02-17,2015,2,17,48,8,KR 5-CL 12 28,...,1.50,0.44,0.0,10.003,320.0,NaN,NaN,NaN,NaN,NaN
2,-74.158180,4.639044,2015-05-07 05:50:00,2015-05-07,2015,5,7,127,5,AV AVENIDA CIUDAD DE CALI-CL 2 S 02,...,3.10,0.44,0.0,10.003,41.0,NaN,NaN,NaN,NaN,NaN
3,-74.164141,4.633424,2015-05-07 05:00:00,2015-05-07,2015,5,7,127,5,AV AVENIDA CIUDAD DE CALI-CL 38D S 42,...,3.10,0.44,0.0,10.003,41.0,NaN,NaN,NaN,NaN,NaN
4,-74.088759,4.591854,2015-03-08 01:50:00,2015-03-08,2015,3,8,67,1,AV AVENIDA CARACAS-CL 1 02,...,1.50,0.44,0.0,10.003,80.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,-74.130809,4.643545,2015-01-21 07:00:00,2015-01-21,2015,1,21,21,7,CL 12-KR 71F 02,...,4.11,0.44,0.0,10.003,41.0,NaN,NaN,NaN,NaN,NaN
60851,-74.117114,4.691121,2015-10-12 21:00:00,2015-10-12,2015,10,12,285,21,CL 65-TR 93 02,...,4.60,0.44,2.0,10.003,230.0,NaN,NaN,NaN,NaN,NaN
60852,-74.112710,4.554628,2015-09-16 08:00:00,2015-09-16,2015,9,16,259,8,KR 5L-CL 48P S 02,...,3.60,0.44,0.0,10.003,41.0,NaN,NaN,NaN,NaN,NaN
60853,-74.105655,4.495597,2015-06-28 11:00:00,2015-06-28,2015,6,28,179,11,CL 97C-KR 2B SE 02,...,2.09,0.75,0.0,10.003,30.0,NaN,NaN,NaN,NaN,NaN


In [16]:
df_18_1 = df_18[df_18['sample_type'] == 1]
df_18_0 = df_18[df_18['sample_type'] == 0]
df_19_1 = df_19[df_19['sample_type'] == 1]
df_19_0 = df_19[df_19['sample_type'] == 0]

In [17]:
df_18_1.head(50)

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,...,windSpeed,cloudCover,uvIndex,visibility,windBearing,windGust,pressure,precipType,ozone,precipAccumulation
0,-74.121790,4.603106,2018-03-25 12:10:00,2018-03-25,2018,3,25,84,12,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,...,2.09,0.44,0.0,10.003,30.0,NaN,NaN,NaN,NaN,NaN
2,-74.178939,4.624160,2018-02-13 07:45:00,2018-02-13,2018,2,13,44,7,KR 86-CL 53 S 2,...,2.60,0.44,0.0,10.003,341.0,NaN,NaN,NaN,NaN,NaN
3,-74.180517,4.598093,2018-02-13 07:00:00,2018-02-13,2018,2,13,44,7,CL 65-KR 77H S 2,...,2.60,0.44,0.0,10.003,341.0,NaN,NaN,NaN,NaN,NaN
6,-74.121020,4.721842,2018-05-02 17:50:00,2018-05-02,2018,5,2,122,17,CL 80-KR 114 99,...,5.70,0.75,7.0,10.003,101.0,NaN,NaN,NaN,NaN,NaN
12,-74.065706,4.599421,2018-04-26 09:00:00,2018-04-26,2018,4,26,116,9,AV AVENIDA CIRCUNVALAR-CL 17 02,...,3.60,0.75,0.0,10.003,50.0,NaN,NaN,NaN,NaN,NaN
13,-74.094294,4.600074,2018-06-19 12:00:00,2018-06-19,2018,6,19,170,12,CL 4B-KR 23A 28,...,2.09,0.44,1.0,10.003,330.0,NaN,NaN,NaN,NaN,NaN
14,-74.081688,4.605078,2018-06-19 12:20:00,2018-06-19,2018,6,19,170,12,CL 13-KR 16 02,...,2.09,0.44,1.0,10.003,330.0,NaN,NaN,NaN,NaN,NaN
16,-74.079973,4.600711,2018-06-18 14:55:00,2018-06-18,2018,6,18,169,14,KR 12-CL 10 49,...,4.60,0.75,4.0,10.003,161.0,NaN,NaN,NaN,NaN,NaN
18,-74.077495,4.665175,2018-07-18 10:30:00,2018-07-18,2018,7,18,199,10,CL 68-KR 51 02,...,3.60,0.75,0.0,10.003,60.0,NaN,NaN,NaN,NaN,NaN
19,-74.179553,4.603938,2018-07-06 07:15:00,2018-07-06,2018,7,6,187,7,CL 59-KR 78D S 2,...,2.09,0.44,0.0,10.003,170.0,NaN,NaN,NaN,NaN,NaN


In [18]:
df_18_0.reset_index(inplace = True)
df_19_0.reset_index(inplace = True)

In [19]:
df_18_0 = df_18_0[df_18_0.summary.isna() == False]
df_19_0 = df_19_0[df_19_0.summary.isna() == False]

In [20]:
df_18_0 = df_18_0.head(df_18_1.shape[0])
df_19_0 = df_19_0.head(df_19_1.shape[0])

In [21]:
df_18 = pd.concat([df_18_1, df_18_0], sort = False)
df_19 = pd.concat([df_19_1, df_19_0], sort = False)

In [22]:
print(df_15.shape)
print(df_16.shape)
print(df_17.shape)
print(df_18.shape)
print(df_19.shape)

(60855, 38)
(66453, 38)
(68191, 38)
(66230, 41)
(58106, 39)


In [23]:
df = pd.concat([df_15, df_16, df_17, df_18, df_19], sort = False)
print(df.shape)

(319835, 41)


In [24]:
df.sample_type.value_counts()

1    159919
0    159916
Name: sample_type, dtype: int64

In [25]:
# df['accident_type'].unique()

In [26]:
df_1 = df[df['sample_type'] == 1]
df_0 = df[df['sample_type'] == 0]

In [27]:
df_1.isna().sum()

x                           0
y                           0
timestamp                   0
date                        0
year                        0
month                       0
day                         0
day_of_year                 0
hour                        0
address                     0
severity                    0
accident_type               0
severity_numeric            0
borough                     0
cluster_id                  0
sample_type                 0
area_km2                    0
population                  0
population_density          0
timestamp_darksky           0
summary                 10044
icon                    10044
precipIntensity         10010
precipProbability       10010
temperature                13
apparentTemperature        13
dewPoint                    0
humidity                   13
windSpeed                 657
cloudCover                151
uvIndex                    57
visibility                158
windBearing             10053
windGust  

In [28]:
df_0.isna().sum()

x                           0
y                           0
timestamp                   0
date                        0
year                        0
month                       0
day                         0
day_of_year                 0
hour                        0
address                     0
severity               159916
accident_type          159916
severity_numeric       159916
borough                     0
cluster_id                  0
sample_type                 0
area_km2                    0
population                  0
population_density          0
timestamp_darksky           0
summary                  1479
icon                     1479
precipIntensity          1449
precipProbability        1449
temperature                 0
apparentTemperature         0
dewPoint                    0
humidity                    0
windSpeed                  80
cloudCover                 53
uvIndex                     9
visibility                 79
windBearing              9891
windGust  

In [29]:
df.isna().sum()

x                           0
y                           0
timestamp                   0
date                        0
year                        0
month                       0
day                         0
day_of_year                 0
hour                        0
address                     0
severity               159916
accident_type          159916
severity_numeric       159916
borough                     0
cluster_id                  0
sample_type                 0
area_km2                    0
population                  0
population_density          0
timestamp_darksky           0
summary                 11523
icon                    11523
precipIntensity         11459
precipProbability       11459
temperature                13
apparentTemperature        13
dewPoint                    0
humidity                   13
windSpeed                 737
cloudCover                204
uvIndex                    66
visibility                237
windBearing             19944
windGust  

In [30]:
df.columns

Index(['x', 'y', 'timestamp', 'date', 'year', 'month', 'day', 'day_of_year',
       'hour', 'address', 'severity', 'accident_type', 'severity_numeric',
       'borough', 'cluster_id', 'sample_type', 'area_km2', 'population',
       'population_density', 'timestamp_darksky', 'summary', 'icon',
       'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'cloudCover', 'uvIndex', 'visibility', 'windBearing', 'windGust',
       'pressure', 'precipType', 'ozone', 'precipAccumulation', 'Unnamed: 0',
       'Unnamed: 0.1', 'index'],
      dtype='object')

In [31]:
to_remove = ['severity', 'accident_type', 'severity_numeric', 'precipType', 'precipAccumulation',
            'Unnamed: 0', 'Unnamed: 0.1', 'ozone', 'pressure', 'windGust', 'index', 'windBearing']

In [32]:
df.drop(columns = to_remove, inplace = True)

In [39]:
df.head()

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,...,cloudCover,uvIndex,visibility,day_of_week,xhour,yhour,xdow,ydow,xmonth,ymonth
0,-74.023482,4.766878,2015-10-17 21:00:00,2015-10-17,2015,10,17,290,21,KR 3-CL 189C 02,...,0.44,2.0,10.003,5,-0.707107,0.707107,-0.974928,-0.222521,-0.866025,0.500000
1,-74.072893,4.598232,2015-02-17 08:10:00,2015-02-17,2015,2,17,48,8,KR 5-CL 12 28,...,0.44,0.0,10.003,1,0.866025,-0.500000,0.781831,0.623490,0.866025,0.500000
2,-74.158180,4.639044,2015-05-07 05:50:00,2015-05-07,2015,5,7,127,5,AV AVENIDA CIUDAD DE CALI-CL 2 S 02,...,0.44,0.0,10.003,3,0.965926,0.258819,0.433884,-0.900969,0.500000,-0.866025
3,-74.164141,4.633424,2015-05-07 05:00:00,2015-05-07,2015,5,7,127,5,AV AVENIDA CIUDAD DE CALI-CL 38D S 42,...,0.44,0.0,10.003,3,0.965926,0.258819,0.433884,-0.900969,0.500000,-0.866025
4,-74.088759,4.591854,2015-03-08 01:50:00,2015-03-08,2015,3,8,67,1,AV AVENIDA CARACAS-CL 1 02,...,0.44,0.0,10.003,6,0.258819,0.965926,-0.781831,0.623490,1.000000,0.000000


In [34]:
# df['weekday'] = df['timestamp'].apply(lambda x: x['timestamp'].dt.dayofweek, axis = 1)
df['day_of_week'] = pd.to_datetime(df['timestamp']).dt.dayofweek


In [38]:
df['xhour'] = round(np.sin(2*np.pi*df['hour']/24),8)
df['yhour'] = round(np.cos(2*np.pi*df['hour']/24),8)
df['xdow'] = round(np.sin(2*np.pi*df['day_of_week']/7),8)
df['ydow'] = round(np.cos(2*np.pi*df['day_of_week']/7),8)
df['xmonth'] = round(np.sin(2*np.pi*df['month']/12),8)
df['ymonth'] = round(np.cos(2*np.pi*df['month']/12),8)

In [36]:
df.columns

Index(['x', 'y', 'timestamp', 'date', 'year', 'month', 'day', 'day_of_year',
       'hour', 'address', 'borough', 'cluster_id', 'sample_type', 'area_km2',
       'population', 'population_density', 'timestamp_darksky', 'summary',
       'icon', 'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'cloudCover', 'uvIndex', 'visibility', 'day_of_week', 'xhour', 'yhour',
       'xdow', 'ydow', 'xmonth', 'ymonth'],
      dtype='object')

In [ ]:
# spacetime_list = df.columns.tolist()[:16]
# spacetime_list.extend(['day_of_week'])
# weather_list = df.columns.tolist()[16:]
# weather_list.remove('day_of_week')

In [ ]:
# spacetime_list

In [ ]:
# weather_list

In [40]:
df.dropna(inplace = True)

In [ ]:
df_weather = df[weather_list]
df_weather

In [ ]:
# weather_list_to_use = ['summary', 'precipIntensity', 'temperature', 'humidity', 'windSpeed', 'visibility']

In [ ]:
# print(df_weather.summary.unique())
# print(df_weather.icon.unique())

In [41]:
#one-hot encoding of categoric variables
df = pd.concat([df, pd.get_dummies(df.summary)], axis=1)
# df = pd.concat([df, pd.get_dummies(df.day_of_week, prefix = 'day')], axis=1)
# df = pd.concat([df, pd.get_dummies(df.month, prefix = 'month')], axis=1)
# df = pd.concat([df, pd.get_dummies(df.hour, prefix = 'hour')], axis=1)


In [42]:
df.drop(columns = ['summary', 'month', 'hour', 'day_of_week'], inplace = True)

In [ ]:
df.columns

In [43]:
label = 'sample_type'

In [44]:
def specificity(y_real, y_predicted):
    tn = pd.np.sum(((y_real == y_predicted) & (y_real == 0)) * 1)
    fp = pd.np.sum((y_real != y_predicted) & (y_predicted == 1) * 1)
    return  tn / (tn + fp)

In [45]:
def scores(predictions, predictions_proba, test, label, predictors):
    
    metrics = {}
    
    # metrics['start_date'] = f1
    # metrics['end_date'] = f2
    metrics['variables used'] = predictors
    metrics['specificity'] = specificity(test[label], predictions)
    metrics['accuracy'] = accuracy_score(test[label], predictions)
    metrics['precision'] = precision_score(test[label], predictions)
    metrics['recall'] = recall_score(test[label],predictions)
    metrics['F1_score'] = f1_score(test[label], predictions)
    metrics['false_positives'] = confusion_matrix(test[label], predictions).ravel()[1]
    metrics['true_positives'] = confusion_matrix(test[label], predictions).ravel()[3]
    metrics['false_negatives'] = confusion_matrix(test[label], predictions).ravel()[2]
    metrics['true_negatives'] = confusion_matrix(test[label], predictions).ravel()[0]
    metrics['roc_auc_score'] = roc_auc_score(test[label], predictions_proba)
    
    return metrics


In [ ]:
df.summary.unique()

In [46]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Model attempt #1

In [ ]:
df.columns

In [47]:
to_ignore = ['timestamp', 'date', 'borough', 'year', 'day', 'day_of_year', 'address', 'cluster_id', 'population', 'timestamp_darksky'
            , 'icon', 'precipṔrobability', 'apparentTemperature', 'dewPoint', 'cloudCover', 'uvIndex', 'sample_type']

In [ ]:
# predictors_weather_1 = ['precipIntensity', 'temperature', 'humidity', 'windSpeed', 'visibility']
# predictors_weather_1.extend(df_weather.summary.unique())

In [ ]:
# predictors_weather_1

In [ ]:
# spacetime_list_1 = ['x','y', 'hour_0', 'day_of_year', 'area_km2', 'population_density']

In [ ]:
# predictors_1 = []
# predictors_1.extend(spacetime_list_1)
# predictors_1.extend(predictors_weather_1)


In [ ]:
# predictors_1

In [69]:
predictors_1 = predictors_1[:16]

In [70]:
# predictors_1 = [x for x in df.columns.tolist() if x not in to_ignore]
all_1 = predictors_1.copy()
all_1.append(label)
# all_1
# all_1 = [x for x in df.columns.tolist() if x not in to_ignore]
         # and x != label]

In [71]:
all_1

['x',
 'y',
 'area_km2',
 'population_density',
 'precipIntensity',
 'precipProbability',
 'temperature',
 'humidity',
 'windSpeed',
 'visibility',
 'xhour',
 'yhour',
 'xdow',
 'ydow',
 'xmonth',
 'ymonth',
 'sample_type']

In [83]:
# all_1 = ['x', 'y', 'population_density', 'xhour', 'yhour', 'xdow', 'ydow', 'xmonth', 'ymonth', 'sample_type']

In [84]:
df_model_1 = df[all_1]

In [85]:
train_1, test_1 = train_test_split(df_model_1, 
                               test_size=0.3,
                               stratify=df_model_1[label],
                               random_state=42)

In [86]:
train_1

,x,y,population_density,xhour,yhour,xdow,ydow,xmonth,ymonth,sample_type
40020,-74.093592,4.726260,12117.27,0.707107,-0.707107,0.000000,1.000000,-0.866025,0.500000,0
59269,-74.081844,4.671487,20459.24,-0.965926,-0.258819,0.000000,1.000000,0.500000,0.866025,0
28350,-74.123954,4.683756,11858.41,0.000000,-1.000000,0.000000,1.000000,1.000000,0.000000,1
20275,-74.104353,4.685239,24723.52,-0.707107,-0.707107,-0.781831,0.623490,0.500000,0.866025,1
4282,-74.102926,4.594884,15225.65,0.866025,-0.500000,0.781831,0.623490,0.000000,-1.000000,1
...,...,...,...,...,...,...,...,...,...,...
42514,-74.095953,4.626659,14921.25,0.258819,-0.965926,0.781831,0.623490,0.866025,0.500000,1
14438,-74.068774,4.616136,2436.30,-0.866025,0.500000,-0.974928,-0.222521,0.500000,-0.866025,0
49772,-74.077273,4.609317,15225.65,-0.500000,-0.866025,-0.974928,-0.222521,-0.500000,0.866025,0
25141,-74.093793,4.625778,14921.25,-0.965926,0.258819,-0.974928,-0.222521,-0.500000,-0.866025,1


In [87]:
print(train_1.sample_type.value_counts())
print(test_1.sample_type.value_counts())

0    110906
1    104912
Name: sample_type, dtype: int64
0    47531
1    44963
Name: sample_type, dtype: int64


In [93]:
train_1.columns.tolist()[:-1]

['x',
 'y',
 'population_density',
 'xhour',
 'yhour',
 'xdow',
 'ydow',
 'xmonth',
 'ymonth']

In [107]:
 
# predictors_1= train_1.columns.tolist()[:-1]

In [108]:
predictors_1

['x',
 'y',
 'population_density',
 'xhour',
 'yhour',
 'xdow',
 'ydow',
 'xmonth',
 'ymonth']

In [95]:
X_train_1 = train_1[predictors_1] 
X_test_1 = test_1[predictors_1] 


y_train_1 = train_1[label]
y_test_1 = test_1[label]

In [98]:
X_train_1

40020    0
59269    0
28350    1
20275    1
4282     1
        ..
42514    1
14438    0
49772    0
25141    1
17003    1
Name: sample_type, Length: 215818, dtype: int64

In [100]:
%%time
rf_1 = RandomForestClassifier(n_estimators=150, max_features=9, random_state=1, n_jobs=-1)
rf_1.fit(X_train_1, y_train_1)

CPU times: user 6min 42s, sys: 2.32 s, total: 6min 45s
Wall time: 58.4 s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=-1, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [99]:
%%time
rf_1 = RandomForestClassifier(n_estimators=150, max_features=10, random_state=1, n_jobs=-1)
rf_1.fit(X_train_1, y_train_1)

ValueError: max_features must be in (0, n_features]

In [109]:
rf_predictions_1 = rf_1.predict(test_1[predictors_1])
rf_predictions_proba_1 = rf_1.predict_proba(test_1[predictors_1])[:,1]

In [110]:
# rf_predictions_1_train = rf_1.predict(train_1[predictors_1])
# rf_predictions_proba_1_train = rf_1.predict_proba(train_1[predictors_1])[:,1]
rf_predictions_1_train = rf_1.predict(train_1[predictors_1])
rf_predictions_proba_1_train = rf_1.predict_proba(train_1[predictors_1])[:,1]

In [111]:
rf_scores_1_train = scores(rf_predictions_1_train, rf_predictions_proba_1_train, train_1, label, predictors_1)
rf_scores_1_train

<ipython-input-44-b38378081b5c>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  tn = pd.np.sum(((y_real == y_predicted) & (y_real == 0)) * 1)
<ipython-input-44-b38378081b5c>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  fp = pd.np.sum((y_real != y_predicted) & (y_predicted == 1) * 1)


{'variables used': ['x',
  'y',
  'population_density',
  'xhour',
  'yhour',
  'xdow',
  'ydow',
  'xmonth',
  'ymonth'],
 'specificity': 0.997258940003246,
 'accuracy': 0.9969047994143213,
 'precision': 0.9971006752374776,
 'recall': 0.9965304254994662,
 'F1_score': 0.9968154688125703,
 'false_positives': 304,
 'true_positives': 104548,
 'false_negatives': 364,
 'true_negatives': 110602,
 'roc_auc_score': 0.9999774318742024}

In [112]:
rf_scores_1 = scores(rf_predictions_1, rf_predictions_proba_1, test_1, label, predictors_1)
rf_scores_1

<ipython-input-44-b38378081b5c>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  tn = pd.np.sum(((y_real == y_predicted) & (y_real == 0)) * 1)
<ipython-input-44-b38378081b5c>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  fp = pd.np.sum((y_real != y_predicted) & (y_predicted == 1) * 1)


{'variables used': ['x',
  'y',
  'population_density',
  'xhour',
  'yhour',
  'xdow',
  'ydow',
  'xmonth',
  'ymonth'],
 'specificity': 0.5348088615850708,
 'accuracy': 0.489847990139901,
 'precision': 0.4735350841686707,
 'recall': 0.44231924026421726,
 'F1_score': 0.4573951841034015,
 'false_positives': 22111,
 'true_positives': 19888,
 'false_negatives': 25075,
 'true_negatives': 25420,
 'roc_auc_score': 0.48567772596398306}

In [82]:
#Importance of variables:
rf_feature_importances_1 = pd.DataFrame(rf_1.feature_importances_,
                                   index = train_1[predictors_1].columns,
                                   columns=['importance']).sort_values('importance',ascending=False)
rf_feature_importances_1

,importance
y,0.201317
x,0.195569
temperature,0.089054
windSpeed,0.085535
humidity,0.080392
ymonth,0.045205
yhour,0.044715
xdow,0.043444
xhour,0.042275
xmonth,0.038777


In [ ]:
rf_feature_importances_1

In [ ]:
df[df['day_of_year']>365].sample_type.nunique()

# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [ ]:
#%%time

param_grid= {
        'min_child_weight': [2, 5],
        'gamma': [0.5],
        'subsample': [0.5, 0.8],
        'colsample_bytree': [0.6, 0.8],
        'max_depth': [8,10],
        'n_estimators': [300],
        'learning_rate':[0.1]
        }

modelXG = XGBClassifier(objective='binary:logistic', silent=True, nthread=1)
random_search_XG = RandomizedSearchCV(estimator = modelXG, param_distributions = param_grid, cv = 2, scoring='roc_auc', n_jobs = -1, verbose = 2, n_iter=100)
random_search_XG.fit(X_train_1, y_train_1)

In [ ]:
random_search_XG.best_params_

In [ ]:
model = random_search_XG.best_estimator_

In [ ]:
xgb_predicted_1 = model.predict(test_1[predictors_1])
xgb_predicted_proba_1 = model.predict_proba(test_1[predictors_1])[:,1]

In [ ]:
xgb_predicted_1_train = model.predict(train_1[predictors_1])
xgb_predicted_proba_1_train = model.predict_proba(train_1[predictors_1])[:,1]

In [ ]:
xgb_scores_1_train = scores(xgb_predicted_1_train, xgb_predicted_proba_1_train, train_1, label, predictors_1)
xgb_scores_1_train

In [ ]:
xgb_scores_1 = scores(xgb_predicted_1, xgb_predicted_proba_1, test_1, label, predictors_1)
xgb_scores_1